# Data Wrangling: Join, Combine, and Reshape

In many cases, data may be spread across several files or databases or be arranged in a form that is not easy to analyse. This note focuses on tools to help you combine, join and rearrange data. 

# Hierarchical Indexing

Recall the concept of _hierarchical indexing_ in pandas that was introduced in one of my earlier notes. _Hierarchical indexing_ is an important feature of pandas that enables you to have multiple (two or more) index levels on an axis and would be used extensively in some of these data wrangling operations. In an abstract sense, it provides a way for you to work with higher dimensional data in a lower dimension form. Let's start with a simple example; create a Series with a list of lists as the index:

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.Series(np.random.randn(9),
                index = [['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.092095
   2   -0.733330
   3    0.328353
b  1    0.487660
   3    0.911450
c  1   -0.224917
   2   -1.042312
d  2    0.237193
   3    0.996111
dtype: float64

With a hierarchical indexed object, so-called _partial_ indexing is possible, enabling you to concisely select subsets of the data:

In [3]:
data['b']

1    0.48766
3    0.91145
dtype: float64

In [4]:
data['b':'c']

b  1    0.487660
   3    0.911450
c  1   -0.224917
   2   -1.042312
dtype: float64

In [5]:
data.loc[['b', 'd']]

b  1    0.487660
   3    0.911450
d  2    0.237193
   3    0.996111
dtype: float64

In [6]:
# selection from an "inner" level
data.loc[:, 2]

a   -0.733330
c   -1.042312
d    0.237193
dtype: float64

Hierarchical indexing plays an important role in reshaping data and group-based operations like forming a pivot table. For example, you could rearrange the data into a DataFrame using its `unstack` method:

In [7]:
data.unstack()  # unpacks hierarchical-index Series into a DataFrame

,1,2,3
a,0.092095,-0.733330,0.328353
b,0.487660,NaN,0.911450
c,-0.224917,-1.042312,NaN
d,NaN,0.237193,0.996111


The reverse operation of `unstack` is `stack`:

In [8]:
data.unstack().stack()

a  1    0.092095
   2   -0.733330
   3    0.328353
b  1    0.487660
   3    0.911450
c  1   -0.224917
   2   -1.042312
d  2    0.237193
   3    0.996111
dtype: float64

`stack` and `unstack` will be explored in more detail later in this note.

With a DataFrame, either axis can have a hierarchical index:

In [9]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                    index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                    columns=[['Ohio', 'Ohio', 'Colorado'], 
                            ['Green', 'Red', 'Green']])

frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

The hierarchical levels can have names (as strings or any Python objects). If so, these will show up in the console output:

In [10]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

With partial column indexing, you can similarly select groups of columns:

In [11]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

### Reordering and Sorting Levels

At times, you will need to rearrange the order of the levels on an axis or sort the data by the values in one specific level. The `swaplevel` takes two level numbers or names and returns a new object with the levels interchanged (but the data is otherwise unaltered):

In [12]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

`sort_index`, on the other hand, sorts the data using only the values in a single level. When swapping levels, it's not uncommon to also use `sort_index` so that the result is lexigraphically sorted by the indicated level:

In [13]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [14]:
frame.swaplevel(0,1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

_**Note:** Data selection performance is much beter on hierarchically indexed objects if the index is lexicographically sorted starting with the outermost level - that is, the result of calling `sort_index(level=0)` or `sort_index()`_

### Summary Statistics by Level

Many descriptive and summary statistics on DataFrame and Series have a `level` option in which you can specify the level you want to aggregate by on a particular axis. Consider the above DataFrame; we can aggregate by level on either the rows or columns like so:

In [15]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [16]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

Under the hood, this utilises pandas's `groupby` mechanics, which will be discussed in more detail later in this note.

### Indexing with a DataFrame's columns

It's not unusual to want to use one or more columns froa DataFrame as the row index; alternatively, you may wish to move the row index into the DataFrame's columns. Here's an example DataFrame:

In [17]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})

frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


DataFrame's `set_index` function will create a new DataFrame using one or more of its columns as the index:

In [18]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

By default, the columns are rmeoved from the DataFrame, though you can leave them in:

In [19]:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

`reset_index`, on the other hand, does the opposite of `set_index`; the hierarchical index levels are moved into the columns:

In [20]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## Combining and Merging Datasets

Data contained in pandas objects can be combined together in a number of ways:

+ `pandas.merge`  connects rows in DataFrames based on one or more keys. This will be familiar to users of SQL or other relational databases, as it implements database _join_ operations.
+ `pandas.concat` concatenates or "stacks" together objects along an axis. 
+ The `combine_first` instance method enables splicing together overlapping data to fill in missing values in one object with values from another.

These will be addressed alongside a number of example later in this note. 

### Database-Style DataFrame Joins

_Merge_ or _join_ operations combine datasets by linking rows using one or more keys. These operations are central to relational databases (e.g. SQL-based). The _merge_ function in pandas is the main entry point for using these algorithms on your data. 

Let's start with a simple example:

In [21]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})

df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [22]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


This is an example of a _many-to-one_ join; the data in `df1` has multiple rows labeled `a` and `b`, whereas `df2` has only one row for each value in the `key` column. Calling `merge` with these objects we obtain: 

In [23]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


Note that I didn't specify which column to join on. If that information is not specified, `merge` uses the overlapping column names as the keys. It's a good practice to specify explicitly, though:

In [24]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


If the column names are different in each object, you can specify them separately:

In [31]:
# rename column label
df3 = df1.rename(columns={'key': 'lkey'})
df4 = df2.rename(columns={'key': 'rkey'})

pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


Notice that the 'c' and 'd' values and their associated data are missing from the result. By default, `merge` does an 'inner' join: the keys in the result are the intersection, or the common set found in both tables. Other possible options are `'left'`, `'right'`, and `'outer'`. The `outer` join takes the union of the keys, combining the effect of applying both left and right joins:

In [32]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


Table below shows a summary of the options for `how`:

**Option** | **Behaviour**
--- | ---
`'inner'` | Use only the key combinations observed in both tables
`'left'` | Use all key combinations found in the left table
`'right'` |  Use all key combinations found in the right table
`'outer'` |  Use all key combinations observed in both tables together

_Many-to-many_ merges have well-defined, though not necessarily intuitive, behaviour. Here's an example:

In [34]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})

df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})


df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [35]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [36]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


Many-to-many joins form the Cartesian product of the rows. Since there were three 'b' rows in the left DataFrame and two in the right one, there are six 'b' rows in the result. The `join` method only affects the distinct key values appearing in the result:

In [37]:
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


To merge with multiple keys, pass a list of column names: 

In [38]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})

right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})

pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


**Note:** When you're joining columns-on-columns, the indexes on the passed DataFrame objects are discarded.

A last issue to consider in merge operations is the treatment of overlapping column names. While you can address the overlap manually, `merge` has a `suffixes` option for specifying strings to append to overlapping names in the left and right DataFrame objects:

In [39]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [40]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


See table below for an argunment reference on `merge`. Joining using the DataFrame's row index is the subject of the next section.

**Argument** | **Description**
--- | ---
`left` | DataFrame to be merged on the left side.
`right` | DataFrame to be merged on the right side. 
`how` | One of `'inner'`, `'outer'`, `'left'`, or `'right'`; defaults to `'inner'`.
`on` | Column names to join on. Must be found in both DataFrame objects. If not specified and no other join keys given, will use the intersection of the column names in `left` and `right` as the join keys.
`left-on` | Columns in `left` DataFrame to use as join keys.
`right_on` | Analogous to `left_on` for `left` DataFrame.
`left_index` | Use row index in `left` as its join key
`right_index` | Analogous to `left_index`.
`sort` | Sort merged data lexicographically by join keys; `True` by default (disable to get better performance in some cases on large datasets).
`suffixes` | Tuple of string values to append to column names in case of overlap; defaults to ('\_x', '\_y') (e.g. if `data` in both DataFrame objects, would appear as `data_x` and `data_y` in result)
`copy` | If `False`, avoid copying data into resulting data structure in some exceptional cases; by default always copies
`indicator` | Adds a special column `_merge` that indicates the source of each row; values will be `'left_only'`, `'right_only'`, or `'both'` based on the origin of the joined data in each row.

### Merging on Index

In some cases, the merge key(s) in a DataFrame will be found in its index. In this case, you can pass `left_index=True` or `right_index=True` (or both) to indicate that the index should be used as the merge key: